# TIME ARIMA

In [ ]:
import itertools

import numpy as np
from statsmodels.tsa.arima.model import ARIMA

from utils import time_plot, plot_arma_process, stationarity_tests, get_figure, acf_plot, pacf_plot
from data import get_apple_5y, get_switzerland_temperature

## Autoregressive process

In [ ]:
np.random.seed(42)
# AR coeffs Note: coefficients are in the form of the characteristic polynomial
# first coefficient for the zero-lag term
ar_parameters = [
    ([1, 1/7], '$x_t = -\\frac{1}{7}x_{t-1}+w_t$'),
    ([1, -3/2, 1/2], '$x_t = \\frac{3}{2}x_{t-1}-\\frac{1}{2}x_{t-2}+w_t$'),
    ([1, -5/6, -1/6, 1/6], '$x_t = \\frac{5}{6}x_{t-1}+\\frac{1}{6}x_{t-2}-\\frac{1}{6}x_{t-3}+w_t$'),
    ([1, 0, -5/4, 0, 1/4], '$x_t = \\frac{5}{4}x_{t-2}-\\frac{1}{4}x_{t-4}+w_t$'),
]
for ar_coeffs, formula in ar_parameters:
    plot_arma_process(ar_coeffs, [1], formula, acf_or_pacf=True)

### Partial Autocorrelation Function (PACF)

In [ ]:
np.random.seed(42)
for ar_coeffs, formula in ar_parameters:
    plot_arma_process(ar_coeffs, [1], formula, acf_or_pacf=False)

## Moving Average process

In [ ]:
np.random.seed(42)
# MA coeffs Note: coefficients are in the form of the characteristic polynomial
# first coefficient for the zero-lag term
ma_parameters = [
    ([1, -1/2], '$x_t = w_t-\\frac{1}{2}w_{t-1}$'),
    ([1, -1, -3/4], '$x_t = w_t-w_{t-1}-\\frac{3}{4}w_{t-2}$'),
    ([1, -1/2, -1/9, 1/18], '$x_t = w_t-\\frac{1}{2}w_{t-1}-\\frac{1}{9}w_{t-2}+\\frac{1}{18}w_{t-3}$'),
    ([1, -1, -3, 1, 2], '$x_t = w_t-w_{t-1}-3w_{t-2}+w_{t-3}+2w_{t-4}$'),
]
for ma_coeffs, formula in ma_parameters:
    plot_arma_process([1], ma_coeffs, formula, acf_or_pacf=True)

### Partial Autocorrelation Function (PACF)

In [ ]:
np.random.seed(42)
for ma_coeffs, formula in ma_parameters:
    plot_arma_process([1], ma_coeffs, formula, acf_or_pacf=False)

## ARMA process

In [ ]:
np.random.seed(42)
nsamples = 1000
arma_parameters = list(itertools.product(ar_parameters, ma_parameters))
for (ar_coeffs, ar_formula), (ma_coeffs, ma_formula) in arma_parameters:
    formula = f'${ar_formula.strip("$").split("w_t")[0]}{ma_formula.strip("$").split("=")[1]}$'
    plot_arma_process(ar_coeffs, ma_coeffs, formula, acf_or_pacf=True)

### Partial Autocorrelation Function (PACF)

In [ ]:
np.random.seed(42)
for (ar_coeffs, ar_formula), (ma_coeffs, ma_formula) in arma_parameters:
    formula = f'${ar_formula.strip("$").split("w_t")[0]}{ma_formula.strip("$").split("=")[1]}$'
    plot_arma_process(ar_coeffs, ma_coeffs, formula, acf_or_pacf=False)

## Example: Apple closing prices
### Stationarity

In [ ]:
data = get_apple_5y()
data = data.resample('W').mean().ffill()
differenced_data = data['Close'].diff().dropna()
stationarity_tests(differenced_data)
fig, [ax1, ax2] = get_figure(ncols=2)
time_plot(
    x=data.index,
    y=data['Close'],
    title='Weekly average of Apple Closing Prices - Last 5 Years',
    xlabel='Date',
    ylabel='Closing Price (USD)',
    ax=ax1,
)
time_plot(
    x=differenced_data.index,
    y=differenced_data,
    title='Differenced data',
    ylabel='$\\nabla x_t$',
    ax=ax2,
)

### ACF and PACF

In [ ]:
fig, [ax1, ax2] = get_figure(ncols=2)
acf_plot(differenced_data, 30, ax1, title="Differenced data")
pacf_plot(differenced_data, 30, ax2, title="Differenced data")

### Fit ARIMA model

In [ ]:
model = ARIMA(data['Close'], order=(0,1,1))
model_fit = model.fit()
print(model_fit.summary())
forecast_steps = 26
forecast = model_fit.forecast(steps=forecast_steps)

fig, ax = time_plot(
    x=data.index,
    y=data['Close'],
    title='Forecast of Monthly Average Apple Closing Prices',
    xlabel='Date',
    ylabel='Closing Price (USD)',
    return_fig=True,
)
ax.plot(forecast.index, forecast, label="ARIMA(0,1,1)", color='red')
ax.legend()

## Example Swiss temperatures
### Stationarity

In [ ]:
data = get_switzerland_temperature()
data = data.set_index('dt')
data = data.asfreq('ME')
differenced_data = data['AverageTemperature'].diff(12).dropna()
stationarity_tests(differenced_data)
fig, [ax1, ax2] = get_figure(ncols=2)
time_plot(
    x=data.index,
    y=data['AverageTemperature'],
    title='Monthly Average Temperature in Switzerland',
    xlabel='Year',
    ylabel='Average Temperature (°C)',
    ax=ax1,
)
time_plot(
    x=differenced_data.index,
    y=differenced_data,
    title='Differenced data',
    ylabel='$\\nabla x_t$',
    ax=ax2,
)

### ACF and PACF

In [ ]:
fig, [ax1, ax2] = get_figure(ncols=2)
acf_plot(differenced_data, 30, ax1, title="Differenced data")
pacf_plot(differenced_data, 30, ax2, title="Differenced data")

### Fit ARIMA model

In [ ]:
model = ARIMA(
    data['AverageTemperature'], 
    order=(0,0,0),
    seasonal_order=(1,1,0,12),
)
model_fit = model.fit()
print(model_fit.summary())
forecast_steps = 26
forecast = model_fit.forecast(steps=forecast_steps)

fig, ax = time_plot(
    x=data.index,
    y=data['AverageTemperature'],
    title='Forecast of Monthly Average Temperature in Switzerland',
    xlabel='Year',
    ylabel='Average Temperature (°C)',
    return_fig=True,
)
ax.plot(forecast.index, forecast, label="ARIMA(0,0,0)(1,1,0)12", color='red')
ax.legend()